# **Progetto: Modello per l'Identificazione della Lingua di Testi per un Museo**

## **Problema**

La gestione delle informazioni multilingue è una sfida significativa per il museo. Attualmente, la lingua di ogni descrizione deve essere identificata manualmente, il che richiede tempo e può portare a errori. Con l'espansione delle collezioni, il numero di testi da analizzare aumenta, rendendo necessario un approccio automatizzato e scalabile.

## **Obiettivo del progetto:**

L'obiettivo è sviluppare un modello di machine learning basato su tecniche di Natural Language Processing (NLP) per identificare la lingua di testi forniti dal museo. Questo modello dovrà:

- Riconoscere automaticamente la lingua di un testo.
- Supportare almeno 3 lingue principali.
- Essere facile da integrare con il sistema esistente del museo.

## **Benefici del progetto:**

- Automazione: Eliminare la necessità di identificazione manuale delle lingue.
- Efficienza: Processare rapidamente grandi volumi di testi.
- Accuratezza: Ridurre gli errori umani nell'identificazione delle lingue.

---

## **Codice:**


### **Librerie**
<a name = "librerie" />

In [ ]:
# Librerie

import nltk
nltk.download('stopwords')

stopwords_it = set(stopwords.words('italian'))
stopwords_en = set(stopwords.words('english'))
stopwords_de = set(stopwords.words('german'))

all_stopwords = stopwords_it | stopwords_en | stopwords_de

import pandas as pd
import re
import string
import numpy as np
from nltk import ngrams
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, log_loss
from scipy.stats import uniform, randint

RANDOM_SEED = 42

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **Funzioni**
<a name = "funzioni" />

In [ ]:
def data_cleaner(sentence): # Funzione per la pulizia del testo

    sentence = sentence.lower() # Tolgo le maiuscole
    for c in string.punctuation: # Itero per togliere la punteggiatura e metto uno spazio
        sentence = sentence.replace(c, " ")

    sentence = ' '.join(word for word in sentence.split() if word not in all_stopwords) # Rimuovo le stopwords
    sentence = re.sub("\d", "", sentence) # Tolgo i numeri
    sentence = re.sub(" +", " ", sentence) # Tolgo gli spazzi multipli

    return sentence

def bow_count(dataset, count_vectorizer): # Funzione per applicare il CountVectorizer

    X = count_vectorizer.transform(dataset) # Trasformo una frase

    return X.toarray()


def pred(model, sentence, cvectorized): # Funzione per analizzare del testo inserito dell'utente

    try:
        cleaned_sentence = data_cleaner(sentence) # Pulisco la frase
        ngrams_list = [" ".join(["".join(gram) for gram in ngrams(cleaned_sentence, 3)])] # Creo i tri-grammi
        vectorized_sentence = bow_count(ngrams_list, cvectorized) # Trasformo la frase
        risultato = model.predict(vectorized_sentence) # Faccio la predizione

        # Decodifico il numero nella lingua corrispondente
        lingua_predetta = label_encoder.inverse_transform([risultato[0]])[0]

        return f"La frase da te inserita corrisponde alla lingua: {lingua_predetta}"

    except Exception as e:

        return f"Errore durante la predizione: {e}"

### **Main / Codice principale**
<a name = "main-/-codice-principale" />

In [ ]:
df_lingue = pd.read_csv("https://raw.githubusercontent.com/Profession-AI/progetti-ml/refs/heads/main/Modello%20per%20l'identificazione%20della%20lingua%20dei%20testi%20di%20un%20museo/museo_descrizioni.csv") # Carico il dataset
df_lingue

,Testo,Codice Lingua
0,Statua in marmo di un imperatore romano del II...,it
1,Anfora greca con decorazioni a figure nere,it
2,Dipinto rinascimentale raffigurante la Madonna...,it
3,Elmo corinzio in bronzo del VI secolo a.C.,it
4,Manoscritto medievale con miniature dorate,it
...,...,...
289,Wikinger-Anhänger mit Darstellung von Thor,de
290,Päpstliches Bleisiegel mit dem Bild des Heilig...,de
291,Leder-Gürtel mit langobardischer Bronzeschließe,de
292,Renaissance-Kupferbecken mit pflanzlichen Must...,de


In [ ]:
df_lingue["Codice Lingua"].value_counts()

,count
Codice Lingua,
it,98
en,98
de,98


In [ ]:
target = df_lingue["Codice Lingua"] # Valore da prevedere

df_lingue_cleaned = [data_cleaner(sentence) for sentence in df_lingue["Testo"]] # Pulisco il dataset
ngr_df_lingue_cleaned = [" ".join(["".join(gram) for gram in ngrams(sentence, 3)]) for sentence in df_lingue_cleaned] # Creo i tri-grammi e li racchiudo in una sola stringa

cvectorized = CountVectorizer(analyzer="char", ngram_range=(1, 3)) # Creo il CountVectorizer
cvectorized.fit(ngr_df_lingue_cleaned) # Eseguo il fit sul dataset

ngr_bow_df = bow_count(ngr_df_lingue_cleaned, cvectorized) # E lo trasformo

X_train, X_test, y_train, y_test = train_test_split(ngr_bow_df, target, test_size=0.2, stratify=target, random_state=RANDOM_SEED) # Divido il dataset in train e test

In [ ]:
label_encoder = LabelEncoder()  # Inizializza il LabelEncoder per convertire le etichette testuali in numeri
y_train = label_encoder.fit_transform(y_train)  # Adatta il LabelEncoder ai dati di training
y_test = label_encoder.transform(y_test)  # Converte le etichette del test set in numeri usando la stessa codifica

#### Primo modello (MLPClassifier)

In [ ]:
param_distributions = {
    "hidden_layer_sizes": [
        (randint(50, 200).rvs(),),
         (randint(50, 200).rvs(), randint(50, 200).rvs())], # 2 Strutte di hiddenlayer da testare
    "activation": ["relu", "tanh","logistic"], # Vari modelli da testare
    "alpha": uniform(0.0001, 0.01), # Distribuzione uniforme tra 0.0001 e 0.01
    "learning_rate_init": uniform(0.001, 0.01), # Distribuzione uniforme tra 0.001 e 0.01
}

mlp = MLPClassifier(max_iter=300, random_state=RANDOM_SEED, solver="adam", early_stopping=False) # Creo il modello MLPClassifier

rs = RandomizedSearchCV( # Inizializzo il modello per trovare gli iperparametri
    estimator=mlp,
    param_distributions=param_distributions,
    scoring="accuracy",
    refit=True,
    n_iter=20,
    cv=10,
    random_state=RANDOM_SEED,
    n_jobs=-1,
)

In [ ]:
rs.fit(X_train, y_train) # Esecuzione della ricerca degli iperparametri

# Migliori iperparametri e miglior punteggio
print("Migliori iperparametri:", rs.best_params_)
print("Miglior punteggio:", rs.best_score_)

Migliori iperparametri: {'activation': 'relu', 'alpha': 0.009756320330745593, 'hidden_layer_sizes': (131, 100), 'learning_rate_init': 0.004854165025399161}
Miglior punteggio: 0.9704710144927537


In [ ]:
clf = rs.best_estimator_ # Prendo i migliori iperparametri
clf.fit(X_train, y_train) # Addestro il modello sul dataset di train

y_pred_test = clf.predict(X_test) # Prevedo i risultati del test set
print("Classification Report (Test Set):\n", classification_report(y_test, y_pred_test)) # Stampo la tabella dei risultati

y_pred_train = clf.predict(X_train)  # Prevedo i risultati per il training set
print("\nClassification Report (Training Set):\n", classification_report(y_train, y_pred_train)) # Stampo la tabella dei risultati

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       1.00      0.95      0.97        19
           1       0.95      0.95      0.95        20
           2       0.95      1.00      0.98        20

    accuracy                           0.97        59
   macro avg       0.97      0.97      0.97        59
weighted avg       0.97      0.97      0.97        59


Classification Report (Training Set):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        79
           1       1.00      1.00      1.00        78
           2       1.00      1.00      1.00        78

    accuracy                           1.00       235
   macro avg       1.00      1.00      1.00       235
weighted avg       1.00      1.00      1.00       235



#### Secondo modello (Multinomial Naive Bayes)

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_pred_test = mnb.predict(X_test) # Prevedo i risultati il test set
print("Classification Report (Test Set):\n", classification_report(y_test, y_pred_test)) # Stampo la tabella dei risultati

y_pred_train = mnb.predict(X_train)  # Prevedo i risultati per il training set
print("\nClassification Report (Training Set):\n", classification_report(y_train, y_pred_train)) # Stampo la tabella dei risultati

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        19
           1       0.94      0.80      0.86        20
           2       0.87      1.00      0.93        20

    accuracy                           0.92        59
   macro avg       0.92      0.92      0.91        59
weighted avg       0.92      0.92      0.91        59


Classification Report (Training Set):
               precision    recall  f1-score   support

           0       1.00      0.99      0.99        79
           1       0.99      0.97      0.98        78
           2       0.97      1.00      0.99        78

    accuracy                           0.99       235
   macro avg       0.99      0.99      0.99       235
weighted avg       0.99      0.99      0.99       235



---

## **Spiegazione del codice:**

### ---> **Sezione: ([Librerie Utilizzate](#librerie))**

Questa sezione contiene l'importazione delle principali **librerie Python** necessarie per il nostro progetto.  
Le librerie incluse coprono la manipolazione dei dati, il preprocessing del testo, la visualizzazione, l'addestramento di modelli di machine learning e metriche di valutazione.

#### 🔹 Librerie Importate:
- **pandas** → Per la manipolazione dei dataset  
- **re, string** → Per la gestione e la manipolazione di stringhe e espressioni regolari  
- **numpy** → Per operazioni matematiche e gestione di array  
- **nltk**:
  - `ngrams` → Per la generazione di n-grammi  
  - `stopwords` → Per la gestione delle stopwords in diverse lingue (`italiano, inglese, tedesco`)  
- **scikit-learn**:
  - `LabelEncoder` → Per l'encoding delle etichette di classe  
  - `CountVectorizer` → Per la trasformazione del testo in rappresentazioni numeriche  
  - `train_test_split, RandomizedSearchCV` → Per la suddivisione dei dati e l'ottimizzazione degli iperparametri  
  - `MLPClassifier` → Per la creazione di una rete neurale artificiale  
  - `MultinomialNB` → Per il modello Naive Bayes multinomiale  
  - `classification_report, accuracy_score, log_loss` → Per la valutazione delle prestazioni del modello  
- **scipy.stats** → Per la generazione di distribuzioni casuali e ricerca degli iperparametri  

#### Codice:
```python
import nltk
nltk.download('stopwords')

stopwords_it = set(stopwords.words('italian'))
stopwords_en = set(stopwords.words('english'))
stopwords_de = set(stopwords.words('german'))

all_stopwords = stopwords_it | stopwords_en | stopwords_de

import pandas as pd
import re
import string
import numpy as np
from nltk import ngrams
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, log_loss
from scipy.stats import uniform, randint

RANDOM_SEED = 42


---

### ---> **Sezione: Funzioni per la Pulizia e Predizione del Testo ([Funzioni Utilizzate](#funzioni))**

Questa sezione contiene tre funzioni essenziali per la gestione del testo e la predizione della lingua.

</br>

#### 🔹 1. Pulizia del Testo (`data_cleaner`)
Questa funzione:
- Converte il testo in minuscolo.
- Rimuovo le stopwords
- Rimuove punteggiatura e numeri.
- Sostituisce spazi multipli con uno solo.

```python
def data_cleaner(sentence): # Funzione per la pulizia del testo

    sentence = sentence.lower() # Tolgo le maiuscole
    for c in string.punctuation: # Itero per togliere la punteggiatura e metto uno spazio
        sentence = sentence.replace(c, " ")

    sentence = ' '.join(word for word in sentence.split() if word not in all_stopwords)
    sentence = re.sub("\d", "", sentence) # Tolgo i numeri
    sentence = re.sub(" +", " ", sentence) # Tolgo gli spazzi multipli

    return sentence
```

---

#### 🔹 2. Conversione in Bag of Words (`bow_count`)
Trasforma il testo in una rappresentazione numerica con `CountVectorizer`.

```python
def bow_count(dataset, count_vectorizer): # Funzione per applicare il CountVectorizer

    X = count_vectorizer.transform(dataset) # Trasformo una frase

    return X.toarray()
```

---

#### 🔹 3. Predizione della Lingua (`pred`)
Questa funzione prende in input una frase e restituisce la lingua predetta dal modello. Ecco i passaggi principali:  
1. **Pulizia del testo** → La frase viene normalizzata con `data_cleaner`.  
2. **Creazione dei trigrammi** → Il testo viene trasformato in sequenze di tre caratteri consecutivi.  
3. **Vettorizzazione** → I trigrammi vengono convertiti in una rappresentazione numerica con `bow_count`.  
4. **Predizione con il modello ML** → Il modello prevede l'etichetta numerica corrispondente alla lingua.  
5. **Decodifica dell'etichetta** → Il numero viene convertito nel nome della lingua (es. "it", "en", "de") usando `LabelEncoder`.  

```python
def pred(model, sentence, cvectorized, label_encoder):  # Aggiunto label_encoder come parametro
    try:
        cleaned_sentence = data_cleaner(sentence)  # Pulisco la frase
        ngrams_list = [" ".join(["".join(gram) for gram in ngrams(cleaned_sentence, 3)])]  # Creo i tri-grammi
        vectorized_sentence = bow_count(ngrams_list, cvectorized)  # Trasformo la frase
        risultato = model.predict(vectorized_sentence)  # Faccio la predizione

        # Decodifico il numero nella lingua corrispondente
        lingua_predetta = label_encoder.inverse_transform([risultato[0]])[0]

        return f"La frase da te inserita corrisponde alla lingua: {lingua_predetta}"

    except Exception as e:
        return f"Errore durante la predizione: {e}"

```

---

### ---> **Sezione: Descrizione del Codice per l'Identificazione della Lingua dei Testi ([Codice principale](#main-/-codice-principale))**

In questo codice, il nostro obiettivo è costruire un modello di machine learning per identificare la lingua dei testi di un museo.

#### 🔹 1. **Caricamento dei Dati**  
   Il codice inizia caricando un dataset da un file CSV presente su GitHub. Il dataset contiene descrizioni di opere e il loro corrispondente codice di lingua. La riga di codice:

   ```python
   df_lingue = pd.read_csv("https://raw.githubusercontent.com/.../museo_descrizioni.csv")
   ```
   Legge i dati e li carica in un dataframe di pandas.

---

#### 🔹 2. **Preprocessing del Testo**  
   Il preprocessing del testo è una fase fondamentale per la preparazione dei dati. In questa sezione, ogni descrizione del museo viene trasformata in un formato adatto all'analisi del modello di machine learning.  

   **Passaggi principali:**  
   1. **Selezione del target** → La colonna `"Codice Lingua"` viene estratta come valore da prevedere, rappresentando la lingua della descrizione.  
   2. **Pulizia del testo** → Ogni frase viene normalizzata e ripulita utilizzando la funzione `data_cleaner`, che rimuove caratteri indesiderati e standardizza il testo.  
   3. **Generazione dei trigrammi** → Dopo la pulizia, ogni frase viene trasformata in un insieme di **tri-grammi**, ovvero sequenze di tre caratteri consecutivi.  
   4. **Unione dei trigrammi** → I trigrammi generati vengono poi concatenati in un'unica stringa per facilitare la successiva vettorizzazione.  

   **Codice:**  
   ```python
   target = df_lingue["Codice Lingua"]  # Valore da prevedere (etichetta della lingua)

   df_lingue_cleaned = [data_cleaner(sentence) for sentence in df_lingue["Testo"]]  # Pulizia del testo
   ngr_df_lingue_cleaned = [" ".join(["".join(gram) for gram in ngrams(sentence, 3)]) for sentence in df_lingue_cleaned]  # Creazione e unione dei tri-grammi
   ```


---

#### 🔹 3. **Estrazione delle Caratteristiche**  
   Viene utilizzato il **CountVectorizer** di `scikit-learn` per trasformare le frasi in una rappresentazione numerica basata su tri-grammi.  
   Questo processo crea una matrice di frequenze, che rappresenta la presenza di n-grammi nei testi.
   
   ```python
   cvectorized = CountVectorizer(analyzer='char', ngram_range=(1, 3))
   cvectorized.fit(ngr_df_lingue_cleaned)
   ```

---

#### 🔹 4. **Divisione del Dataset in Set di Addestramento e Test**   
  Per garantire una corretta valutazione del modello, il dataset viene suddiviso in due insiemi:  
  - **Training set** (`X_train` e `y_train`) → Utilizzato per addestrare il modello.  
  - **Test set** (`X_test` e `y_test`) → Utilizzato per valutare le prestazioni del modello su dati mai visti prima.  

La suddivisione avviene in modo **stratificato** grazie al parametro `stratify=target`. Questo assicura che la distribuzione delle lingue sia simile in entrambi i set, evitando sbilanciamenti che potrebbero compromettere l'addestramento del modello.  

  **Codice:**  
   ```python
   X_train, X_test, y_train, y_test = train_test_split(ngr_bow_df, target, test_size=0.2, stratify=target, random_state=RANDOM_SEED)  # Divido il dataset mantenendo la distribuzione originale delle classi
   ```

---

#### 🔹 5. **Ricerca degli Iperparametri Ottimali**  
   Utilizzando il `RandomizedSearchCV`, viene eseguita una ricerca casuale per trovare i migliori iperparametri per un modello di **Multi-Layer Perceptron (`MLP`)**,  
   che è una rete neurale completamente connessa. I parametri ottimizzati includono la struttura del layer nascosto, la funzione di attivazione, l’alpha e la velocità di apprendimento.

   ```python
   rs = RandomizedSearchCV(estimator=mlp, param_distributions=param_distributions, scoring="accuracy", refit=True, n_iter=20, cv=10, random_state=RANDOM_SEED, n_jobs=-1)
   ```

---

#### 🔹 6. **Addestramento del 1° Modello (MLPClassifier)**  
   Una volta trovati i migliori iperparametri, il modello viene addestrato con i dati di addestramento.

   ```python
   clf = rs.best_estimator_
   clf.fit(X_train, y_train)
   ```

---

#### 🔹 7. **Valutazione del 1° Modello (MLPClassifier)**  
   Dopo l’addestramento, il modello è testato sui dati di test e train e viene generato un **classification report** per valutare le sue performance,  
   includendo metriche come precision, recall e F1-score.

   ```python
   y_pred_test = clf.predict(X_test) # Prevedo i risultati il test set
   print("Classification Report (Test Set):\n", classification_report(y_test, y_pred_test)) # Stampo la tabella dei risultati

   y_pred_train = clf.predict(X_train)  # Prevedo i risultati per il training set
   print("\nClassification Report (Training Set):\n", classification_report(y_train, y_pred_train)) # Stampo la tabella dei risultati
   ```

---

### 🔹 8. **Addestramento del 2° Modello (Multinomial Naive Bayes)**  
   In aggiunta al modello basato su reti neurali, viene addestrato un **Naive Bayes Multinomial** (`MultinomialNB`).  
   Questo modello è spesso efficace per la classificazione di testi e utilizza la distribuzione multinomiale per calcolare la probabilità delle classi.

   ```python
   mnb = MultinomialNB()
   mnb.fit(X_train, y_train)
   ```

---

### 🔹 9. **Valutazione del 2° Modello (Multinomial Naive Bayes)**  
   Il modello viene testato sui dati di test e train e viene generato un **classification report** per valutare le sue performance.  

   ```python
   y_pred_test = mnb.predict(X_test) # Prevedo i risultati il test set
   print("Classification Report (Test Set):\n", classification_report(y_test, y_pred_test)) # Stampo la tabella dei risultati

   y_pred_train = mnb.predict(X_train)  # Prevedo i risultati per il training set
   print("\nClassification Report (Training Set):\n", classification_report(y_train, y_pred_train)) # Stampo la tabella dei risultati
   ```



---

## **Test:**

### 1. (MLPClassifier)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(clf, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: Dies ist eine antike griechische Amphora, die etwa im 5. Jahrhundert v. Chr. gefertigt wurde. Die Vase besteht aus Terrakotta und zeigt filigrane schwarzfigurige Malerei, die Szenen aus dem täglichen Leben und der Mythologie darstellt. Ihre elegante, schlanke Form ist typisch für diese Periode, und sie wurde wahrscheinlich zur Aufbewahrung von Flüssigkeiten wie Wein oder Öl verwendet. Die detaillierte Kunst auf dem Gefäß gibt einen Einblick in die kulturellen und künstlerischen Praktiken des antiken Griechenlands.
La frase da te inserita corrisponde alla lingua: de


---

### 2. (MLPClassifier)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(clf, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: This is an ancient Greek amphora, crafted around the 5th century BCE. The vase is made from terracotta and features intricate black-figure pottery depicting scenes from daily life and mythology. Its elegant, slender shape is characteristic of the period, and it was likely used for storing liquids such as wine or oil. The detailed artwork on the vessel provides a glimpse into the cultural and artistic practices of ancient Greece.
La frase da te inserita corrisponde alla lingua: en


---

### 3. (MLPClassifier)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(clf, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: Questa è un'antica anfora greca, realizzata intorno al V secolo a.C. La vase è fatta di terracotta e presenta intricate pitture a figure nere che raffigurano scene della vita quotidiana e della mitologia. La sua forma elegante e slanciata è caratteristica del periodo, ed era probabilmente utilizzata per conservare liquidi come vino o olio. L'arte dettagliata sull'oggetto offre uno spunto sulle pratiche culturali e artistiche della Grecia antica.
La frase da te inserita corrisponde alla lingua: it


---

### 4. (Multinomial Naive Bayes)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(mnb, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: Questa è un'antica anfora greca, realizzata intorno al V secolo a.C. La vase è fatta di terracotta e presenta intricate pitture a figure nere che raffigurano scene della vita quotidiana e della mitologia. La sua forma elegante e slanciata è caratteristica del periodo, ed era probabilmente utilizzata per conservare liquidi come vino o olio. L'arte dettagliata sull'oggetto offre uno spunto sulle pratiche culturali e artistiche della Grecia antica.
La frase da te inserita corrisponde alla lingua: it


---

### 5. (Multinomial Naive Bayes)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(mnb, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: This is an ancient Greek amphora, crafted around the 5th century BCE. The vase is made from terracotta and features intricate black-figure pottery depicting scenes from daily life and mythology. Its elegant, slender shape is characteristic of the period, and it was likely used for storing liquids such as wine or oil. The detailed artwork on the vessel provides a glimpse into the cultural and artistic practices of ancient Greece.
La frase da te inserita corrisponde alla lingua: en


---

### 6. (Multinomial Naive Bayes)

In [ ]:
print(f"Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {set(df_lingue['Codice Lingua'])}")

sentence = input("Inserisci una frase da analizzare: ")
risultato = pred(mnb, sentence, cvectorized)

print(risultato)

Questo è il programma per analizzare e riconoscere la lingua delle descrizioni, supporta le seguenti lingue: {'en', 'de', 'it'}
Inserisci una frase da analizzare: Dies ist eine antike griechische Amphora, die etwa im 5. Jahrhundert v. Chr. gefertigt wurde. Die Vase besteht aus Terrakotta und zeigt filigrane schwarzfigurige Malerei, die Szenen aus dem täglichen Leben und der Mythologie darstellt. Ihre elegante, schlanke Form ist typisch für diese Periode, und sie wurde wahrscheinlich zur Aufbewahrung von Flüssigkeiten wie Wein oder Öl verwendet. Die detaillierte Kunst auf dem Gefäß gibt einen Einblick in die kulturellen und künstlerischen Praktiken des antiken Griechenlands.
La frase da te inserita corrisponde alla lingua: de


---